<a href="https://colab.research.google.com/github/sggyuan/GNN/blob/main/%E6%9C%80%E7%BB%88%EF%BC%81%EF%BC%81%EF%BC%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.4.0+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 33.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.nn import GAE
import torch.nn.functional as F

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = torch.load('/content/drive/MyDrive/my_graph_data-bal.pt')

<ipython-input-4-e9945f3ad513>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load('/content/drive/MyDrive/my_graph_data-bal.pt')


In [5]:
data = train_test_split_edges(data)

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [6]:
data

Data(x=[16283, 13], y=[100000], val_pos_edge_index=[2, 5000], val_pos_edge_attr=[5000, 8], test_pos_edge_index=[2, 10000], test_pos_edge_attr=[10000, 8], train_pos_edge_index=[2, 99256], train_pos_edge_attr=[99256, 8], train_neg_adj_mask=[16283, 16283], val_neg_edge_index=[2, 5000], test_neg_edge_index=[2, 10000])

In [7]:
class GCNEncoder(nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels=64):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, out_channels)

        self.bn1 = nn.BatchNorm1d(hidden_channels)
        self.bn2 = nn.BatchNorm1d(hidden_channels)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x, edge_index):
        # First GCN layer
        x1 = F.leaky_relu(self.bn1(self.conv1(x, edge_index)))
        x1 = self.dropout(x1)

        # Second GCN layer with residual connection
        x2 = F.leaky_relu(self.bn2(self.conv2(x1, edge_index))) + x1
        x2 = self.dropout(x2)

        # Final GCN layer
        x3 = self.conv3(x2, edge_index)

        return x3


In [8]:
out_channels = 2
num_features = data.num_features
epochs = 100

model1 = GAE(GCNEncoder(num_features, out_channels))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model1 = model1.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

optimizer = torch.optim.Adam(model1.parameters(), lr=0.001, weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=10, factor=0.5)


In [34]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model1.encode(x, train_pos_edge_index)
    loss = model1.recon_loss(z, train_pos_edge_index)
    #if args.variational:
    #   loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model1.eval()
    with torch.no_grad():
        z = model1.encode(x, train_pos_edge_index)
    return model1.test(z, pos_edge_index, neg_edge_index)

In [55]:
from __future__ import division

import torch
import torch.nn.functional as F
from torch_scatter import scatter_add


def true_positive(pred, target, num_classes):
    r"""Computes the number of true positive predictions.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`LongTensor`
    """
    out = []
    for i in range(num_classes):
        out.append(((pred == i) & (target == i)).sum())

    return torch.tensor(out)



def true_negative(pred, target, num_classes):
    r"""Computes the number of true negative predictions.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`LongTensor`
    """
    out = []
    for i in range(num_classes):
        out.append(((pred != i) & (target != i)).sum())

    return torch.tensor(out)



def false_positive(pred, target, num_classes):
    r"""Computes the number of false positive predictions.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`LongTensor`
    """
    out = []
    for i in range(num_classes):
        out.append(((pred == i) & (target != i)).sum())

    return torch.tensor(out)


def false_negative(pred, target, num_classes):
    r"""Computes the number of false negative predictions.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`LongTensor`
    """
    out = []
    for i in range(num_classes):
        out.append(((pred != i) & (target == i)).sum())

    return torch.tensor(out)


def recall(pred, target, num_classes):
    r"""Computes the recall
    :math:`\frac{\mathrm{TP}}{\mathrm{TP}+\mathrm{FN}}` of predictions.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`Tensor`
    """
    tp = true_positive(pred, target, num_classes).to(torch.float)
    fn = false_negative(pred, target, num_classes).to(torch.float)

    out = tp / (tp + fn)
    out[torch.isnan(out)] = 0

    return out



def f1_score(pred, target, num_classes):
    r"""Computes the :math:`F_1` score
    :math:`2 \cdot \frac{\mathrm{precision} \cdot \mathrm{recall}}
    {\mathrm{precision}+\mathrm{recall}}` of predictions.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`Tensor`
    """
    prec = precision(pred, target, num_classes)
    rec = recall(pred, target, num_classes)

    score = 2 * (prec * rec) / (prec + rec)
    score[torch.isnan(score)] = 0

    return score

In [56]:
def test(pos_edge_index, neg_edge_index):
    model1.eval()
    with torch.no_grad():
        z = model1.encode(x, train_pos_edge_index)
        pos_out = model1.decode(z, pos_edge_index)
        neg_out = model1.decode(z, neg_edge_index)

        pred_scores = torch.cat([pos_out, neg_out], dim=0)

        # 创建真实标签
        y_true = torch.cat([torch.ones(pos_out.size(0)), torch.zeros(neg_out.size(0))], dim=0)

        # 确保 pred_scores 和 y_true 在同一设备上
        pred_scores = pred_scores.cpu()
        y_true = y_true.cpu()

        # 将预测分数转换为二进制预测
        y_pred = (pred_scores > 0.5).long()

        # 计算 AUC 和 AP
        auc = roc_auc_score(y_true.numpy(), pred_scores.numpy())
        ap = average_precision_score(y_true.numpy(), pred_scores.numpy())

        # 计算 Recall 和 F1 分数
        num_classes = 2  # 因为这是二分类问题
        rec = recall(y_pred, y_true, num_classes)[1]  # 只取正类的召回率
        f1 = f1_score(y_pred, y_true, num_classes)[1]  # 只取正类的 F1 分数

    return auc, ap, rec.item(), f1.item()

# 在主循环中使用
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap, rec, f1 = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print(f'Epoch: {epoch:03d}, AUC: {auc:.4f}, AP: {ap:.4f}, '
          f'Recall: {rec:.4f}, F1: {f1:.4f}')

Epoch: 001, AUC: 0.8784, AP: 0.9197, Recall: 0.9075, F1: 0.7176
Epoch: 002, AUC: 0.8786, AP: 0.9199, Recall: 0.9080, F1: 0.7167
Epoch: 003, AUC: 0.8783, AP: 0.9198, Recall: 0.9071, F1: 0.7185
Epoch: 004, AUC: 0.8784, AP: 0.9199, Recall: 0.9074, F1: 0.7185
Epoch: 005, AUC: 0.8784, AP: 0.9200, Recall: 0.9080, F1: 0.7188
Epoch: 006, AUC: 0.8783, AP: 0.9200, Recall: 0.9077, F1: 0.7195
Epoch: 007, AUC: 0.8780, AP: 0.9198, Recall: 0.9067, F1: 0.7207
Epoch: 008, AUC: 0.8776, AP: 0.9197, Recall: 0.9061, F1: 0.7216
Epoch: 009, AUC: 0.8771, AP: 0.9194, Recall: 0.9046, F1: 0.7232
Epoch: 010, AUC: 0.8766, AP: 0.9192, Recall: 0.9033, F1: 0.7243
Epoch: 011, AUC: 0.8767, AP: 0.9192, Recall: 0.9037, F1: 0.7238
Epoch: 012, AUC: 0.8766, AP: 0.9192, Recall: 0.9035, F1: 0.7241
Epoch: 013, AUC: 0.8771, AP: 0.9195, Recall: 0.9045, F1: 0.7228
Epoch: 014, AUC: 0.8776, AP: 0.9197, Recall: 0.9055, F1: 0.7218
Epoch: 015, AUC: 0.8778, AP: 0.9198, Recall: 0.9058, F1: 0.7209
Epoch: 016, AUC: 0.8782, AP: 0.9200, Rec

In [57]:
from torch_geometric.nn import VGAE

In [58]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class VariationalGCNEncoder(nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels=64):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv_mu = GCNConv(hidden_channels, out_channels)
        self.conv_logstd = GCNConv(hidden_channels, out_channels)

        self.bn1 = nn.BatchNorm1d(hidden_channels)
        self.bn2 = nn.BatchNorm1d(hidden_channels)

        self.dropout = nn.Dropout(0.3)

    def forward(self, x, edge_index):
        # First GCN layer
        x1 = F.leaky_relu(self.bn1(self.conv1(x, edge_index)))
        x1 = self.dropout(x1)

        # Second GCN layer with residual connection
        x2 = F.leaky_relu(self.bn2(self.conv2(x1, edge_index))) + x1
        x2 = self.dropout(x2)

        # Final GCN layers for mean and log-standard deviation
        mu = self.conv_mu(x2, edge_index)
        logstd = self.conv_logstd(x2, edge_index)

        return mu, logstd

    def reparameterize(self, mu, logstd):
        if self.training:
            return mu + torch.randn_like(logstd) * torch.exp(logstd)
        else:
            return mu

In [62]:
out_channels = 2
num_features = data.num_features
epochs = 300


model2 = VGAE(VariationalGCNEncoder(num_features, out_channels))  # new line

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model2 = model2.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)
optimizer = torch.optim.Adam(model2.parameters(), lr=0.01, weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=10, factor=0.5)

In [63]:
def train():
    model2.train()
    optimizer.zero_grad()
    z = model2.encode(x, train_pos_edge_index)
    loss = model2.recon_loss(z, train_pos_edge_index)

    loss = loss + (1 / data.num_nodes) * model2.kl_loss()  # new line
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model2.eval()
    with torch.no_grad():
        z = model2.encode(x, train_pos_edge_index)
    return model2.test(z, pos_edge_index, neg_edge_index)

In [ ]:
def train():
    model2.train()
    optimizer.zero_grad()
    z = model2.encode(x, train_pos_edge_index)
    loss = model2.recon_loss(z, train_pos_edge_index)

    loss = loss + (1 / data.num_nodes) * model2.kl_loss()  # new line
    loss.backward()
    optimizer.step()
    return float(loss)

def test(pos_edge_index, neg_edge_index):
    model2.eval()
    with torch.no_grad():
        z = model2.encode(x, train_pos_edge_index)
        pos_out = model2.decode(z, pos_edge_index)
        neg_out = model2.decode(z, neg_edge_index)

        pred_scores = torch.cat([pos_out, neg_out], dim=0)
        y_true = torch.cat([torch.ones(pos_out.size(0)), torch.zeros(neg_out.size(0))], dim=0)

        pred_scores = pred_scores.cpu()
        y_true = y_true.cpu()

        y_pred = (pred_scores > 0.5).long()

        auc = roc_auc_score(y_true.numpy(), pred_scores.numpy())
        ap = average_precision_score(y_true.numpy(), pred_scores.numpy())

        num_classes = 2
        rec = recall(y_pred, y_true, num_classes)[1]
        f1 = f1_score(y_pred, y_true, num_classes)[1]

    return auc, ap, rec.item(), f1.item()

for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap, rec, f1 = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, AUC: {auc:.4f}, AP: {ap:.4f}, '
          f'Recall: {rec:.4f}, F1: {f1:.4f}')